# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234909221967                   -0.50    8.0
  2   -7.250318760757       -1.81       -1.40    1.0
  3   -7.251312865332       -3.00       -2.13    4.0
  4   -7.251299379330   +   -4.87       -2.28    4.0
  5   -7.251332616890       -4.48       -2.76    2.0
  6   -7.251336727352       -5.39       -2.99    2.0
  7   -7.251338767995       -5.69       -3.90    1.0
  8   -7.251338795979       -7.55       -4.30    4.0
  9   -7.251338798566       -8.59       -4.99    2.0
 10   -7.251338798692       -9.90       -5.51    3.0
 11   -7.251338798702      -11.01       -5.81    3.0
 12   -7.251338798704      -11.58       -6.29    2.0
 13   -7.251338798705      -12.60       -6.81    3.0
 14   -7.251338798705      -13.80       -7.05    1.0
 15   -7.251338798705      -14.35       -7.37    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.02773544392666214
[ Info: Arnoldi iteration step 2: normres = 0.5833026752586292
[ Info: Arnoldi iteration step 3: normres = 0.2805887146613025
[ Info: Arnoldi iteration step 4: normres = 0.7329154971285935
[ Info: Arnoldi iteration step 5: normres = 0.5720362529651676
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (5.06e-01, 1.01e-01, 1.65e-01, 1.76e-01, 5.25e-02)
[ Info: Arnoldi iteration step 6: normres = 0.565025442873891
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.56e-01, 1.03e-01, 4.78e-01, 9.20e-02, 3.80e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08999549452510035
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.34e-02, 2.10e-02, 3.46e-02, 2.87e-02, 6.47e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10149721556398027
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (5.75e-04